In [1]:
######################################################################
# Code produced by Ayden McCarthy
# Manuscript Title: "Machine Learning Models Predict Assessment Outcomes 
#                    From Military Physical Employment Standards Via a 
#                    Physical Test Battery"
# Program of Study: PhD Candidacy
# Institution: Macquarie University
# Year: 2024
######################################################################

######################################################################
# Note for Users:
# This code is intended for use within Python JupyterLab.
# It requires data to be set up according to the instructions 
# outlined in the manuscript. Users can follow the code comments to 
# understand each step of the analysis.
# Please ensure that you replace the placeholder CSV file names in 
# the code with the names of your specific data files to run the code 
# successfully.
######################################################################


In [2]:
# Import necessary libraries and modules for all machine learning models produced in the manuscript, not just the ridge, which is useful for installing these in general.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import shap
import xgboost as xgb
import warnings
import time
from itertools import combinations
from matplotlib.ticker import MaxNLocator
from sklearn.exceptions import ConvergenceWarning
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from scipy import stats
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV, RidgeCV, ElasticNetCV, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score, LeaveOneOut, KFold, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_error
from IPython.display import display, HTML
from sklearn.feature_selection import RFECV
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures


KeyboardInterrupt



In [ ]:
# Create HTML for text with black color
html_text = """
<div style='font-size:70px; font-weight:bold; text-align:center; color: black;'>
    Ridge Regression Model
</div>
"""

# Display the HTML in the output cell
HTML(html_text)

In [ ]:
# Load dataset
df = pd.read_csv('Training_Set_Reduced_with_Important_Features.csv') ###Please change to your own dataset.

# Separate features (predictors) and target variable
X = df.drop(columns=['Weight Lifted (Kg)']) ######## Your outcome variable is to be placed where 'Weight Lifted (Kg)' is. This was the lift-to-place results.
y = df['Weight Lifted (Kg)'] ######## Outcome variable

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define hyperparameter grid for Ridge model
param_grid = {
    'alpha': np.logspace(-8, 8, 200)
}

# Initialize GridSearchCV for Ridge model
ridge_search = GridSearchCV(Ridge(), param_grid, cv=LeaveOneOut(), scoring='neg_mean_squared_error', n_jobs=-1)
ridge_search.fit(X_scaled, y)

# Extract the best Ridge model from GridSearchCV
best_ridge = ridge_search.best_estimator_

# Initialize RFECV for feature selection with Ridge model
rfecv = RFECV(estimator=best_ridge, step=1, cv=LeaveOneOut(), scoring='neg_mean_squared_error')
rfecv.fit(X_scaled, y)

# Number of features selected by RFECV
n_features = rfecv.n_features_
selected_features = X.columns[rfecv.support_]
X_selected = X_scaled[:, rfecv.support_]

print(f"Number of features selected by Ridge model: {n_features}")

# Fit Ridge model with the features selected by RFECV
best_ridge.fit(X_selected, y)

# Calculate RMSE for the Ridge model using the selected features
mse_scores = cross_val_score(best_ridge, X_selected, y, scoring='neg_mean_squared_error', cv=LeaveOneOut())
rmse_optimal = np.sqrt(-mse_scores.mean())
print(f"RMSE using optimal {n_features} features in Ridge model: {rmse_optimal}")

# Initialize SHAP Explainer for model interpretation
explainer = shap.Explainer(best_ridge.predict, shap.sample(X_selected, 100))

# Calculate SHAP values for the selected features
shap_values = explainer(X_selected)
shap.summary_plot(shap_values, X_selected, feature_names=selected_features)

# Calculate mean absolute SHAP values to determine feature importance
shap_sum = np.abs(shap_values.values).mean(axis=0)

# Create a DataFrame with feature names and their corresponding SHAP values
feature_importance = pd.DataFrame(list(zip(selected_features, shap_sum)), columns=['Feature', 'SHAP Value'])
feature_importance = feature_importance.sort_values(by='SHAP Value', ascending=False)

print("Feature Importance based on SHAP values:")
print(feature_importance)

# Store results in a dictionary
LOO_model_results = {
    'Ridge': {
        'RMSE Optimal': rmse_optimal,
        'Optimal Features': selected_features,
        'RFECV Support': rfecv.support_,
        'Model': best_ridge,
        'Feature Importance': feature_importance
    }
}

print("Analysis Complete.")

In [ ]:
# Create HTML for text with black color
html_text = """
<div style='font-size:60px; font-weight:bold; text-align:center; color: black;'>
    Save optimal features as a .csv file
</div>
"""

# Display the HTML in the output cell
HTML(html_text)

In [ ]:
# Assuming LOO_model_results is a dictionary where each key is a model name,
# and each value is a dictionary containing model info, including 'Optimal Features'
for model_name, model_info in LOO_model_results.items():
    # Extract the selected features - Check if 'Optimal Features' exists in the dictionary to avoid KeyError
    if 'Optimal Features' in model_info:
        selected_features = model_info['Optimal Features']
    else:
        print(f"No 'Optimal Features' found for {model_name}.")
        continue

    # Add 'Weight Lifted (Kg)' to the list of features
    # Ensure that 'Weight Lifted (Kg)' is not already in the list to avoid duplication
    if 'Weight Lifted (Kg)' not in selected_features: ############ Your outcome variable is to be placed where 'Weight Lifted (Kg)' is. This was the lift-to-place results.
        selected_features_with_target = list(selected_features) + ['Weight Lifted (Kg)'] ######## Your outcome variable is to be placed where 'Weight Lifted (Kg)' is. This was the lift-to-place results.
    else:
        selected_features_with_target = list(selected_features)

    # Filter the original DataFrame based on these features
    # Ensure the features exist in the DataFrame to avoid KeyError
    missing_features = [feature for feature in selected_features_with_target if feature not in df.columns]
    if missing_features:
        print(f"The following features are missing in the DataFrame for {model_name}: {missing_features}")
        continue
    df_filtered = df[selected_features_with_target]

    # Save the filtered data to a CSV file
    # Using a safe string for the filename (replacing spaces and slashes)
    safe_model_name = model_name.replace(' ', '_').replace('/', '_')
    filename = f'{safe_model_name}_optimal_features_data_LOO.csv'
    df_filtered.to_csv(filename, index=False)

    print(f"Saved data for {model_name} with optimal features to CSV.")


In [ ]:
# Create HTML for text with black color
html_text = """
<div style='font-size:60px; font-weight:bold; text-align:center; color: black;'>
    Optimised Ridge Parameters
</div>
"""

# Display the HTML in the output cell
HTML(html_text)

In [ ]:
# Load the data
df_ridge = pd.read_csv('Ridge_optimal_features_data_LOO.csv')  

# Separate features and target variable
X = df_ridge.drop(['Weight Lifted (Kg)'], axis=1)  ######## Your outcome variable is to be placed where 'Weight Lifted (Kg)' is. This was the lift-to-place results.
y = df_ridge['Weight Lifted (Kg)'] ######## Your outcome variable is to be placed where 'Weight Lifted (Kg)' is. This was the lift-to-place results.

# Standardise the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define a range of alpha values to explore
alphas = np.logspace(-10, 10, 200)  # Wide range of alpha values

# Initialise RidgeCV with Leave-One-Out Cross-Validation
ridge_cv = RidgeCV(alphas=alphas, cv=LeaveOneOut(), scoring='neg_mean_squared_error')

# Fit RidgeCV on the data
ridge_cv.fit(X_scaled, y)

# Find the best alpha value
best_alpha = ridge_cv.alpha_
print("Best alpha for Ridge:", best_alpha)

# Train the Ridge model with the best alpha
ridge_best = Ridge(alpha=best_alpha)
ridge_best.fit(X_scaled, y)

# Make predictions and calculate RMSE
y_pred = ridge_best.predict(X_scaled)
rmse = np.sqrt(mean_squared_error(y, y_pred))
print("Root Mean Squared Error for Ridge:", rmse)

# Plot a histogram of the residuals
residuals = y - y_pred
plt.hist(residuals, bins=30, edgecolor='black')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Histogram of Residuals for Ridge LOO')
plt.show()


In [ ]:
# Create HTML for text with black color
html_text = """
<div style='font-size:60px; font-weight:bold; text-align:center; color: black;'>
    Testing Set - Unseen Data
</div>
"""

# Display the HTML in the output cell
HTML(html_text)

In [ ]:
# Load the testing dataset
df_testing = pd.read_csv('Testing_Set.csv') ###Please change to your own dataset.

# Encoding 'Sex' column
#df_testing['Sex'] = df_testing['Sex'].map({'M': 0, 'F': 1})

# Select the same columns as in the training dataset
X_testing = df_testing[X.columns]

# Standardize the features in the testing dataset using the same scaler used for the training data
X_testing_scaled = scaler.transform(X_testing)

# Use the trained Ridge model to make predictions on the testing dataset
y_pred_testing = ridge_best.predict(X_testing_scaled)

# Add the predicted values to the testing dataset
df_testing['Predicted_Weight_Lifted'] = y_pred_testing

# Calculate RMSE for testing data (if true target values are available)
true_y_testing = df_testing['Weight Lifted (Kg)'] ######## Your outcome variable is to be placed where 'Weight Lifted (Kg)' is. This was the lift-to-place results.
rmse_testing = np.sqrt(mean_squared_error(true_y_testing, y_pred_testing))
print("Root Mean Squared Error on Testing Data (Ridge):", rmse_testing)

# Plot the true vs. predicted values with improved styling
plt.figure(figsize=(10, 8))

# Scatter plot for the predicted values
plt.scatter(true_y_testing, y_pred_testing, alpha=0.7, label='Predicted', color='blue', edgecolors='w')

# Scatter plot for the true values
plt.scatter(true_y_testing, true_y_testing, alpha=0.7, label='True', color='red', edgecolors='w')

# Diagonal line indicating perfect predictions
plt.plot([true_y_testing.min(), true_y_testing.max()], [true_y_testing.min(), true_y_testing.max()], 'k--', lw=2, label='Perfect Prediction')

# Styling the plot
plt.style.use('ggplot')  # Using ggplot style for more appealing visuals
plt.xlabel('True Weight Lifted (Kg)', fontsize=14)
plt.ylabel('Predicted Weight Lifted (Kg)', fontsize=14)
plt.title('Ridge LOO True vs. Predicted Weight Lifted (Testing Data)', fontsize=16, fontweight='bold')
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(fontsize=12)

# Display the plot
plt.show()

# Save the predictions and the plot to files
df_testing.to_csv('Testing_Set_Predictions_Ridge_LOO_RFE.csv', index=False)
plt.savefig('True_vs_Predicted_Plot_Ridge_LOO.png', format='png')
